# Dealing with Data Spring 2020 – Class 5

---

# Files and Printing

Often you will need to read data from a file, or write the output of a Python script back to a file. 

We use the `open` function to open the file in the appropriate mode, which takes two arguments: 

1. the name of the file,
2. and the mode. 

> `a_file = open(filename, mode)`

The `mode` is a single letter string that specifies if you are going to be reading from a file, writing to a file, or appending to the end of an existing file. The modes are: 

+ `'r'` : open a file for reading
+ `'w'` : open a file for writing (beware, this will overwrite any previously existing file) 
+ `'a'` : append (write to the end of a file) 

When reading a file, you usually want to iterate through the lines in that file using a `for loop`. Some other common methods for dealing with files are: 

+ `file.read()` : read the entire contents of a file into a string
+ `file.write(some_string)` : writes to the file (note, this doesn't automatically include new lines) 
+ `file.flush()` : write out any buffered writes
+ `file.close()` : close the open file

# Writing a file to disk

In [39]:
# create the file temp.txt, and get it ready for writing

f = open("temp.txt", "w")
f.write("This is my first file! The end!\n")
f.write("Oh wait, I wanted to say something else.")
f.close()

In [40]:
# the command below is one of the IPython "magics" - commands within the notebook unrelated to python
# %magic shows you the list of basic commands and %lsmagic shows you all the super commands

# for more info, check out https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

%more temp.txt

# Reading a file from disk

In [41]:
# open the file for reading

f = open("temp.txt", "r")
content = f.read() # read the full content of the file in memory, as a big string
f.close()

In [42]:
content

'This is my first file! The end!\nOh wait, I wanted to say something else.'

Once we read the file, we have the lines in a big string. Let's process that big string a little bit:

In [43]:
# read the file in the cell above and split the content of the file using the newline character '\n'

lines = content.split("\n")

# iterate through the line variable (it is a list of strings) and then print the length of each line

for line in lines:
    print(line, " ===> ", len(line))

This is my first file! The end!  ===>  31
Oh wait, I wanted to say something else.  ===>  40


In [44]:
# create a file numbers.txt and write the numbers from 0 to 24 there

f = open("numbers.txt", "w")

for num in range(25):
    f.write(str(num)+'\n')
    
f.close()

In [45]:
%more numbers.txt

In [46]:
# now open the file for reading

f = open("numbers.txt", "r")

# and read the full content of the file in memory, as a big string

content = f.read()

f.close()

content

'0\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n'

In [47]:
# here we convert the strings into integers
# we use the conditional to avoid trying to parse the string '' that is at the end of the list

numbers = []
lines = content.split("\n")

for line in lines:
    if len(line) > 0:
        numbers.append(int(line))
    else:
        continue
        
print(numbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [48]:
# Let's clean up

# windows
#!del temp.txt
#!del numbers.txt

# macOS
!rm temp.txt
!rm numbers.txt

---

# Python `os` standard library
Another addition to our file handling toolkit is the `os` library which provides ways to move files, make directories, and gather data about the file system. Like other standard libraries, we need to import it to use it via `import os`

In [49]:
import os

# let's get info about our current working directory - the folder our Python applications are working in
os.getcwd()

'/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5'

In [50]:
# next, let's list everything in the directory

os.listdir()

['Dealing_with_Data_Class5_Solved.ipynb',
 '.DS_Store',
 'raw_census_2010.csv',
 '.ipynb_checkpoints',
 '[YOUR_NAME_HERE]_Class5_Homework.ipynb',
 'Class5_Homework_Solution.ipynb']

In [51]:
# ... and see what our login name is

os.getlogin()

'siegmanA'

In [52]:
# .. and find out the "separate" used in constructing file paths
# every operating system is different, and this value enables your python to be cross-platform 

os.path.sep

'/'

In [53]:
# ... now we can create our own paths for new files - important for creating a "clean" version of source data

dir_list = os.getcwd().split(os.path.sep)
print(dir_list)

['', 'Users', 'siegmanA', 'Desktop', 'NYU-Dealing-With-Data-Spring-2020', 'Class 5']


In [54]:
# ... now let's create an output file in a new sub-folder called Class 7 - tmp

# first - append the new folder name and create a file path string using os.path.sep and the string.join() method

dir_list.append("Class 5 - tmp")
dir_string = os.path.sep.join(dir_list)

# second - create the directory using the file path

os.mkdir(dir_string)

# third - add the file name "tmp2.txt" to the path

dir_list.append("tmp2.txt")
dir_string = os.path.sep.join(dir_list)

print(dir_string)

/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5/Class 5 - tmp/tmp2.txt


In [55]:
# we can now open the file for writing using the absolute path 
# *we could have also used os.chdir("7-tmp") and open("tmp2.txt") but it's better to use absolute paths

file_handle = open(dir_string,"w")

In [56]:
file_handle.write("test file\nsecond line\n")
file_handle.close()

In [57]:
# clean up 

os.chdir("/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5/Class 5 - tmp")
!rm "tmp2.txt"

os.chdir("..")
!rmdir "Class 5 - tmp"

---

# **Putting our Python to work: Exploring and Cleaning the Census Data file**

So far in our class project, we have identified three data files (IRS tax return counts for NYC, US census data for NYC, and NYC film permits data) and we have skipped over the cleaning step to transform them in SQL.

Now, we'll take one step back and begin cleaning them, starting with the smallest file - the US Census data.

Our approach will be as follows:
1. Inspect the data thoroughly to understand its benefits and risks for processing, and what information lies within
* Clean/fix the data to remove any issues that would prevent it from working with SQL, such as weird characters, too many columns, missing data, splitting values into multiple rows, or combining multiple values into a single row
* Structure the data found in the file as a Python native structure so we can manipulate and prepare it for use in SQL
* Migrate our approach to a dedicated Python file

We'll use: file read/write, loops, nested structures and UDFs to do all of this. (Some will be homework.)

---

## **1) Inspect our Data**
First - we need to understand the data in this file - using either Excel or JupyterLab's CSV reader.

Upon inspection (either way, here's what we learn:
* there are 350+ columns
* there are zipcodes for just NYC 
* there are a mix of letter and numbers for values
* we have both percent and numbers values
* it appears to be comma-separated
* there are two column header lines: one with codes, and another with human-readable labels

Next - we inspect with Python in two ways: first we'll look at the raw file as a string, then we'll probe whether the comma-separation is fail safe or not.

In [58]:
# REVIEWING DATA AS A STRING

import os
os.getcwd()

'/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5'

In [59]:
# find the location of the raw_census_2010.csv file you downloaded and open it for reading into a variable

file_handle = open('/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5/raw_census_2010.csv',"r")
census_data = file_handle.read()
file_handle.close()

In [60]:
# print the first 500 characters of the census file

census_data[:500]

'GEO.id,GEO.id2,GEO.display-label,HD01_S001,HD02_S001,HD01_S002,HD02_S002,HD01_S003,HD02_S003,HD01_S004,HD02_S004,HD01_S005,HD02_S005,HD01_S006,HD02_S006,HD01_S007,HD02_S007,HD01_S008,HD02_S008,HD01_S009,HD02_S009,HD01_S010,HD02_S010,HD01_S011,HD02_S011,HD01_S012,HD02_S012,HD01_S013,HD02_S013,HD01_S014,HD02_S014,HD01_S015,HD02_S015,HD01_S016,HD02_S016,HD01_S017,HD02_S017,HD01_S018,HD02_S018,HD01_S019,HD02_S019,HD01_S020,HD02_S020,HD01_S021,HD02_S021,HD01_S022,HD02_S022,HD01_S023,HD02_S023,HD01_S0'

In [61]:
# and the last 500 characters
census_data[-500:]

'4,44.4,1802,19.9,2373,26.2,1878,20.7,281,3.1,104,1.1,59,0.7,508,5.6,49,0.5,33,0.4,297,3.3,82,0.9,47,0.5,19,0.2,28,0.3,35,0.4,10,0.1,25,0.3,4024,100.0,2434,60.5,1035,25.7,1802,44.8,647,16.1,172,4.3,98,2.4,460,11.4,290,7.2,1590,39.5,1331,33.1,507,12.6,131,3.3,824,20.5,403,10.0,1120,27.8,1169,29.1,2.23, ( X ) ,2.83, ( X ) ,4326,100.0,4024,93.0,302,7.0,75,1.7,5,0.1,49,1.1,24,0.6,32,0.7,117,2.7,1.8, ( X ) ,5.4, ( X ) ,4024,100.0,2726,67.7,6370, ( X ) ,2.34, ( X ) ,1298,32.3,2618, ( X ) ,2.02, ( X ) \n'

# **Review Comma-Based Separability**

In the first 500 characters, we see that the census has "code labels" for each column, none of which have funky characters that could trip us up with comma separation. 

We need to work with just that first line, so we'll use readline (which reads one entire line from the file) on the file handle

In [62]:
file_handle = open('/Users/siegmanA/Desktop/NYU-Dealing-With-Data-Spring-2020/Class 5/raw_census_2010.csv',"r")
census_first_line = file_handle.readline()
file_handle.close()
census_first_line

'GEO.id,GEO.id2,GEO.display-label,HD01_S001,HD02_S001,HD01_S002,HD02_S002,HD01_S003,HD02_S003,HD01_S004,HD02_S004,HD01_S005,HD02_S005,HD01_S006,HD02_S006,HD01_S007,HD02_S007,HD01_S008,HD02_S008,HD01_S009,HD02_S009,HD01_S010,HD02_S010,HD01_S011,HD02_S011,HD01_S012,HD02_S012,HD01_S013,HD02_S013,HD01_S014,HD02_S014,HD01_S015,HD02_S015,HD01_S016,HD02_S016,HD01_S017,HD02_S017,HD01_S018,HD02_S018,HD01_S019,HD02_S019,HD01_S020,HD02_S020,HD01_S021,HD02_S021,HD01_S022,HD02_S022,HD01_S023,HD02_S023,HD01_S024,HD02_S024,HD01_S025,HD02_S025,HD01_S026,HD02_S026,HD01_S027,HD02_S027,HD01_S028,HD02_S028,HD01_S029,HD02_S029,HD01_S030,HD02_S030,HD01_S031,HD02_S031,HD01_S032,HD02_S032,HD01_S033,HD02_S033,HD01_S034,HD02_S034,HD01_S035,HD02_S035,HD01_S036,HD02_S036,HD01_S037,HD02_S037,HD01_S038,HD02_S038,HD01_S039,HD02_S039,HD01_S040,HD02_S040,HD01_S041,HD02_S041,HD01_S042,HD02_S042,HD01_S043,HD02_S043,HD01_S044,HD02_S044,HD01_S045,HD02_S045,HD01_S046,HD02_S046,HD01_S047,HD02_S047,HD01_S048,HD02_S048,HD01_S

In [63]:
# let's separate that first line into a list and see exactly how many columns we have

header1_list = census_first_line.split(",")
len(header1_list)

375

We knew it had 350+ and this is too many columns to effectively work with. 

We know from our transform work together that we need to weight up/down the data, which means percentages won't be useful, so we can remove those. 

We can also remove some of the family occupancy data because it was decided to focus on gender, income and ethnicity in the project.

Before we get to removing data, though, we need to put our data into a structure we can slice and dice. In other words, we need to transform our "string" data into a list of lists - a nested data structure where each row is a list, and within that list, each column is a list - a nested data structure.

For example: let's look at this simple 3x3 table:

In [64]:
example_list = [ 
    ["a","b","c"],
    [123,456,789],
    [987,654,321]
    ]

print(example_list[0])
print(example_list[1])
print(example_list[2])

print("row 1, column 3 ==>",example_list[0][2])
print("row 2, column 2 ==>",example_list[1][1])
print("row 3, column 1 ==>",example_list[2][0])

['a', 'b', 'c']
[123, 456, 789]
[987, 654, 321]
row 1, column 3 ==> c
row 2, column 2 ==> 456
row 3, column 1 ==> 987


## **2) Cleaning Up Wonky Data**

But before we can even create a nested structure - we need to be confident we can split the data correctly for every single line.

Unfortunately, CSV data is known to be particularly tricky because sometimes data sources use commas in column labels but surround those column headers with "" because Excel will treat it right. 

Python won't be so forgiving so we need to test for "" in the data - we'll do this using a for loop.

In [65]:
# using a for loop

quote_count = 0

for char in census_data:
    if char == '"':
        quote_count += 1
    else:
        continue
        
print(quote_count)

28


Uh-oh! those quotes spell trouble so now we need to see where that first quote appears and if a comma appears after it.

In [67]:
quote_pos = census_data.find('"')
comma_pos = census_data.find(",",quote_pos)
census_data[quote_pos - 10: comma_pos + 20]

' 18 years,"Number; HOUSEHOLDS BY TYPE - Total households - Family households (families) [7] - Male householder, no wife present","'

We suspect that those ""  in human-readable column headers of the second line of data are hiding "...**,**..." and will cause any nesting using comma-separation to create extra columns.

Let's prove it.

In [68]:
# we need to isolate the second line using "find": 
# the second line is between the first and second \n

first_nl = census_data.find("\n")
second_nl = census_data.find("\n",first_nl+1)
second_line = census_data[first_nl+1:second_nl]

# split the second line using commas to see how many columns we get

len(second_line.split(","))

391

Now that we've proven those will be a problem, we are going to use Python to clean those up via our own User Defined Function for this purpose, because it may appear in another data source, too.

Before we create the UDF, let's describe what we want our function to do: 

1. it will accept a string input
2. it will remove " characters from the input
3. when it finds a , between "" it will replace , characters in the input with a `-`. However, it will NOT change `,` otherwise since it is a CSV file.
4. it will return a string output

And, we will create a test_input and expected_output to test our UDF during development:

In [69]:
def unquotable(input_string): # will remove those , within "" before changing , to \t 
    
    tmp_list = input_string.split('"') # remove " char by splitting the input string into a list using that char

    # remove , char
    for i in range(len(tmp_list)):
        # skip items that start/end with commas as these aren't quoted items & we don't want to remove their commas 
        if (tmp_list[i][0] == ",") or (tmp_list[i][-1] == ","):
            continue
        else: # replace , with - in quoted items
            tmp_list[i] = tmp_list[i].replace(",","-")

    output = ",".join(tmp_list) # rejoin items into a string

    # get rid of any ",," and ",,," that could exist as it will add extra columns
    output = output.replace(",,,",",")
    output = output.replace(",,",",")
    
    return output # return string

In [70]:
# our test data

test_input = 'something,"test string: a,b,c",other thing'
exp_output = 'something,test string: a-b-c,other thing'

test_output = unquotable(test_input) # our testing lines - run the UDF

test_output == exp_output # compare UDF run to expected output

True

In [71]:
# now we try our second_line variable storing the problem row for cleaning 
# proof it works: no " and len after CSV split = 375

new_row = unquotable(second_line)

print('"' in new_row)
print(len(new_row.split(",")))

False
375


It works! Now, let's fix our original input data string, `census_data`

In [72]:
clean_census_data = unquotable(census_data)

---

## **3) Creating a Nested Structure**

Now that we've cleaned up the wonkiness in the data, we can create our nested structure using a `for` loop.

Let's start by reviewing our simple 3x3 table example:

In [34]:
example_list = [ 
    ["a","b","c"],
    [123,456,789],
    [987,654,321]
    ]

print(example_list[0])
print(example_list[1])
print(example_list[2])

print("row 1, column 3 ==>",example_list[0][2])
print("row 2, column 2 ==>",example_list[1][1])
print("row 3, column 1 ==>",example_list[2][0])

['a', 'b', 'c']
[123, 456, 789]
[987, 654, 321]
row 1, column 3 ==> c
row 2, column 2 ==> 456
row 3, column 1 ==> 987


In our example, the data is structured as:

`list[row][column]`

And we will use split commands to do the same for our data, this time creating another UDF.

Again - as before - let's state what it will do and create test data:
1. the UDF will take a data string as input
* it will create a list where each line in the data, identified using `\n`, is an item
* for each item in the first list `list[row]`, it will create a list of columns, using comma-separatiuon (`,`) to identify each item
* it would be nice for the UDF user to specify what character separates rows and columns, separately
* the UDF will return the nested data structure

In [35]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
#@@@@@@@@ UDF nester() @@@@@@@@@

# take a dataset string where each row is separated by input_row_delim and each column is separate by 
# input_col_delim to create a nested object of lists

def nester(input_string,input_row_delim,input_col_delim):

    row_list = input_string.split(input_row_delim) # create a list item for each row in the file using the row delimiter

    nested_data = [] #output var

    # created nested structure to store each column separately list of rows where each row is a list of columns)
    for i in range(len(row_list)): 
        row = row_list[i]
        col = row.split(input_col_delim)
        nested_data.append(col)
    
    return nested_data # return the nested structure

In [36]:
# our test data

test_input = 'r1c1,r1c2,r1c3\nr2c1,r2c2,r2c3\nr3c1,r3c2,r3c3'
exp_output = [
    ['r1c1','r1c2','r1c3'],
    ['r2c1','r2c2','r2c3'],
    ['r3c1','r3c2','r3c3']
    ]

test_output = nester(test_input,'\n',',') # our testing - run the UDF

test_output == exp_output # compare UDF output to expected output

True

Now we can create a nested structure of our census data:

In [37]:
census_struc = nester(clean_census_data,'\n',',') # structure our original string

In [38]:
print(census_struc[2]) # let's explore more

['8600000US06390', '06390', 'ZCTA5 06390', '236', '100.0', '6', '2.5', '9', '3.8', '16', '6.8', '11', '4.7', '7', '3.0', '7', '3.0', '7', '3.0', '13', '5.5', '20', '8.5', '27', '11.4', '22', '9.3', '29', '12.3', '19', '8.1', '20', '8.5', '6', '2.5', '6', '2.5', '8', '3.4', '3', '1.3', '49.4', ' ( X ) ', '200', '84.7', '196', '83.1', '193', '81.8', '55', '23.3', '43', '18.2', '118', '50.0', '2', '0.8', '7', '3.0', '8', '3.4', '4', '1.7', '4', '1.7', '3', '1.3', '2', '0.8', '7', '3.0', '9', '3.8', '17', '7.2', '8', '3.4', '21', '8.9', '4', '1.7', '11', '4.7', '3', '1.3', '4', '1.7', '4', '1.7', '0', '0.0', '49.2', ' ( X ) ', '100', '42.4', '99', '41.9', '96', '40.7', '24', '10.2', '22', '9.3', '118', '50.0', '4', '1.7', '2', '0.8', '8', '3.4', '7', '3.0', '3', '1.3', '4', '1.7', '5', '2.1', '6', '2.5', '11', '4.7', '10', '4.2', '14', '5.9', '8', '3.4', '15', '6.4', '9', '3.8', '3', '1.3', '2', '0.8', '4', '1.7', '3', '1.3', '49.7', ' ( X ) ', '100', '42.4', '97', '41.1', '97', '41.1', '3

---

## **4) Moving from Notebooks to \*.py Files** 

But before you go to do that, we need to start moving the findings of our exploration into a dedicated python file for cleaning the Census data. 

We're making this migration because notebooks are great for exploring data, but as our files and project grow larger, it is simpler to run the Python files outside notebooks AND sometimes very large files can cause our notebooks to crash.

Here's what that new Python file needs to do:

1. read the census data file into a variable
2. clean the data by removing the "...,..." problem using a UDF
3. create a nested data structure
4. remove unwanted columns
5. create a new string from the nested structure
6. write the file to disk

We've already #1, #2, #3, and #6 together, so we'll isolate those below along with comments to do the other work before putting in its own file.

And recall that our UDFs has to be put ahead of the main program to work because the Main Program needs to know what happens in those UDFs.

Below is the exact code that will be put into its own file named `clean_census.py` and we'll run it together.

```python
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#@@@@@  UDF unquotable() @@@@@@@@@@@@@@@
# removes pesky , within "" values before changing , to \t in program

def unquotable(input_string):
    
    # remove " char by splitting the input string into a list using that char
    tmp_list = input_string.split('"')
 
    # remove , char
    for i in range(len(tmp_list)):
        # skip items that start/end with commas as these aren't quoted items
        if (tmp_list[i][0] == ",") or (tmp_list[i][-1] == ","):
            continue
        # replace , with - in quoted items
        else:
            tmp_list[i] = tmp_list[i].replace(",","-")

    # rejoin items into a string
    output = ",".join(tmp_list)

    # get rid of any ",," and ",,," as it will add extra columns
    output = output.replace(",,,",",")
    output = output.replace(",,",",")
    
    # return string
    return output

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
#@@@@@@@@ UDF nester() @@@@@@@@@
# take a dataset string where each row is separated by input_row_delim 
# and each column is separate by input_col_delim to create a nested object of lists

def nester(input_string,input_row_delim,input_col_delim):

    # create a list item for each row in the file using the row delimiter
    row_list = input_string.split(input_row_delim)

    #output var
    nested_data = []
    
    # created nested structure to store each column separately
    # list of rows where each row is a list of columns)
    for i in range(len(row_list)): 
        row = row_list[i]
        col = row.split(input_col_delim)
        nested_data.append(col)
    
    # return the nested structure
    return nested_data


################################################################
##### MAIN PROGRAM #############################################
################################################################

# 1. read the census data file into a variable

file_handle = open('C:\\Users\\colling\\!dwd_spring2019\\classes\\class7\\raw_census_2010.csv',"r")
census_data = file_handle.read()
file_handle.close()

# 2. clean the data by removing the "...,..." problem using a UDF

clean_census_data = unquotable(census_data)

# 3. create a nested data structure with a UDF

nested_census_data = nester(clean_census_data,"\n",",")

# 4. remove unwanted columns using a UDF

# HOMEWORK

# 5. create a new string from the nested structure using a UDF

# HOMEWORK

# 6. write the file to disk

file_handle = open('C:\\Users\\colling\\!dwd_spring2019\\classes\\class7\\clean_census_2010.csv',"w")
file_handle.write('final data var from step #5 goes here')
file_handle.close()
```